In [5]:
import cv2 # openCV 4.5.1
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
import time 

from skimage.io import imread
from skimage.transform import resize 
from PIL import Image, ImageFont, ImageDraw # add caption by using custom font
from collections import deque
# from shufflenet import ShuffleNet

AttributeError: module 'scipy' has no attribute '_lib'

In [ ]:
base_model=keras.applications.mobilenet_v2.MobileNetV2(input_shape=(160,160, 3), include_top=False, classes=2)
thecheckpoint = r"C:\Users\User\FYP\CodenOutput\projectrvd\MobileNetV2\Model_MobileNetV2_RNN_20221026-133134.h5"
model=keras.models.load_model(thecheckpoint)

if "MobileNetV2" in thecheckpoint:
    modelname = "MobileNetV2"
elif "MobileNet" in thecheckpoint:
    modelname = "MobileNet"
elif "EfficientNet" in thecheckpoint:
    modelname = "EfficientNet"
elif "ShuffleNet" in thecheckpoint:
    modelname = "ShuffleNet"
elif "DenseNet" in thecheckpoint:
    modelname = "DenseNet"

In [3]:
input_video="vidinput/input (3).avi"

if ".avi" in input_video:
    extension = ".avi"
elif ".mp4" in input_video:
    extension = ".mp4"

In [4]:
### VIDEO READER

def video_reader(cv2, filename):
    """Load 1 video file. Next, read each frame image and resize as (fps, 160, 160, 3) shape and return frame Numpy array."""
    
    frames=np.zeros((30, 160, 160, 3), dtype=np.float) #> (fps, img size, img size, RGB)
    
    i=0
    print(frames.shape)
    vid=cv2.VideoCapture(filename) # read frame img from video file.
    
    if vid.isOpened():
        grabbed, frame=vid.read() 
    else:
        grabbed=False
    
    frm=resize(frame,(160, 160, 3))
    frm=np.expand_dims(frm, axis=0)
    
    if(np.max(frm)>1):
        frm=frm/255.0 # Scaling
    frames[i][:]=frm
    i+=1
    print('Reading Video')
    
    while i<30:
        grabbed, frame=vid.read()
        frm=resize(frame, (160, 160, 3)) 
        frm=np.expand_dims(frm, axis=0)
        if(np.max(frm)>1):
            frm=frm/255.0
        frames[i][:]=frm
        i+=1
        
    return frames

In [5]:
### PREDICTION OF IMAGE ARRAY

def create_pred_imgarr(base_model, video_frm_ar):
    """Insert base_model(MobileNet) and result of video_reader() function.
    This function extract features from each frame img by using base_model.
    And reshape Numpy array to insert LSTM model : (1, 30, 25600)"""
    video_frm_ar_dim=np.zeros((1, 30, 160, 160, 3), dtype=np.float)
    video_frm_ar_dim[0][:][:]=video_frm_ar #> (1, 30, 160, 160, 3)
     
    # Extract features from each frame img by using base_model(MobileNet)
    pred_imgarr=base_model.predict(video_frm_ar)
    # Reshape features array : (1, fps, 25600)
    pred_imgarr=pred_imgarr.reshape(1, pred_imgarr.shape[0], 25600)
    
    return pred_imgarr #> ex : (1, 30, 25600)

In [6]:
### PREDICT VIOLENCE

def pred_fight(model, pred_imgarr, acuracy=0.9):
    """If accuracy>=input value(ex:0.9), return (Violence)'True'. else, return 'False'.
    ::model:: trained LSTM model (We already load .h5 file in the above.)
    ::pred_imgarr:: (1, 30, 25600) shaped Numpy array. Extracted features.
    ::accuracy:: default value is 0.9"""

    pred_test=model.predict(pred_imgarr) #> Violence(Fight) : [0,1]. Non-Violence(NonFight) : [1,0]
    
    if pred_test[0][1] >= acuracy:
        return True, pred_test[0][1] #> True, Probability of Violence
    
    else:
        return False, pred_test[0][1] #> False, Probability of Violence

In [7]:
### CHECK ABOVE FUNCTION
video_file = input_video
video_from_ar = video_reader(cv2, video_file)
pred_imgarr = create_pred_imgarr(base_model, video_from_ar)
pred_imgarr.shape

preds = pred_fight(model, pred_imgarr, 0.9)
preds

C:\Users\User\AppData\Local\Temp/ipykernel_8956/533712840.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  frames=np.zeros((30, 160, 160, 3), dtype=np.float) #> (fps, img size, img size, RGB)


(30, 160, 160, 3)
Reading Video


C:\Users\User\AppData\Local\Temp/ipykernel_8956/3932966193.py:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  video_frm_ar_dim=np.zeros((1, 30, 160, 160, 3), dtype=np.float)


(True, 0.99750465)

In [8]:
### DETECT VIOLENCE ALL IN ONE FUNCTION

def detect_violence(video):
    """ It contains video_reader(), create_pred_imgarr(), pred_fight() function as all-in-one.
    ::video:: video file (.mp4, .avi, ...)
    
    video_reader() : Read each frame img by using openCV. Resize Numpy array
    create_pred_imgarr() : Extract features from frame img array by using base model(MobileNet)
    pred_fight() : Decide Violence True or False by using trained LSTM model"""
    
    video_frm_ar=video_reader(cv2, video) 
    pred_imgarr=create_pred_imgarr(base_model, video_frm_ar)
    
    time1=int(round(time.time()*1000))

    f, precent=pred_fight(model, pred_imgarr, acuracy=0.65)
    
    time2=int(round(time.time()*1000))
    
    result={'Violence': f, #> True(Violence), False(Non-Violence)
            'Violence Estimation': str(precent), # Probability of Violence
            'Processing Time' : str(time2-time1)} 
    
    return result

In [9]:
video_file=input_video
detect_violence(video_file)

C:\Users\User\AppData\Local\Temp/ipykernel_8956/533712840.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  frames=np.zeros((30, 160, 160, 3), dtype=np.float) #> (fps, img size, img size, RGB)


(30, 160, 160, 3)
Reading Video


C:\Users\User\AppData\Local\Temp/ipykernel_8956/3932966193.py:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  video_frm_ar_dim=np.zeros((1, 30, 160, 160, 3), dtype=np.float)


{'Violence': True,
 'Violence Estimation': '0.99750465',
 'Processing Time': '133'}

In [10]:
### ADD CAPTION & SAVE VIDEO OUTPUT FILE

input_path = input_video
input_path = input_path.replace(".mp4","")
input_path = input_path.replace(".avi","")
input_path = input_path.replace("vidinput/","")

output_path = f'vidoutput/{input_path}_{modelname}_{time.strftime("%Y%m%d_%H%M%S")}_output{extension}'
print(output_path)

vidoutput/input (3)_DenseNet_20220825_170416_output.avi


In [11]:
vid=cv2.VideoCapture(input_video)
fps=vid.get(cv2.CAP_PROP_FPS) # recognize frames per secone(fps) of input_path video file.
# fps = 30
print(f'fps : {fps}') # print fps.

writer=None
(W, H)=(None, None)
i=0 # number of seconds in video = The number of times that how many operated while loop .
Q=deque(maxlen=128) 

video_frm_ar=np.zeros((1, int(fps), 160, 160, 3), dtype=np.float) #frames
frame_counter=0 # frame number in 1 second. 1~30
frame_list=[] 
preds=None
maxprob=None

#. While loop : Until the end of input video, it read frame, extract features, predict violence True or False.
# ----- Reshape & Save frame img as (30, 160, 160, 3) Numpy array  -----
while True: 
    frame_counter+=1
    grabbed, frm=vid.read() # read each frame img. grabbed=True, frm=frm img. ex: (240, 320, 3)
    
    if not grabbed:
        print('There is no frame. Streaming ends.')
        break

    if W is None or H is None: # W: width, H: height of frame img
        (H, W)=frm.shape[:2]
            
    output=frm.copy() # It is necessary for streaming captioned output video, and to save that.
    
    frame=resize(frm, (160, 160, 3)) #> Resize frame img array to (160, 160, 3)
    frame_list.append(frame) # Append each frame img Numpy array : element is (160, 160, 3) Numpy array.
        
    if frame_counter>=fps: # fps=30 et al
        #. ----- we'll predict violence True or False every 30 frame -----
        #. ----- Insert (1, 30, 160, 160, 3) Numpy array to LSTM model ---
        #. ----- We'll renew predict result caption on output video every 1 second. -----
        # 30-element-appended list -> Transform to Numpy array -> Predict -> Initialize list (repeat)
        frame_ar=np.array(frame_list, dtype=np.float16) #> (30, 160, 160, 3)
        frame_list=[] # Initialize frame list when frame_counter is same or exceed 30, after transforming to Numpy array.
            
        if(np.max(frame_ar)>1): 
            frame_ar=frame_ar/255.0 # Scaling RGB value in Numpy array
        
        pred_imgarr=base_model.predict(frame_ar) #> Extract features from each frame img by using MobileNet. (30, 5, 5, 1024)
        pred_imgarr_dim=pred_imgarr.reshape(1, pred_imgarr.shape[0], 25600)#> (1, 30, 25600)

        preds=model.predict(pred_imgarr_dim) #> (True, 0.99) : (Violence True or False, Probability of Violence)
        print(f'preds:{preds}')
        Q.append(preds)
    
        # Predict Result : Average of Violence probability in last 5 second
        if i<5:
            results=np.array(Q)[:i].mean(axis=0)
        else:
            results=np.array(Q)[(i-5):i].mean(axis=0)
        
        print(f'Results = {results}') #> ex : (0.6, 0.650)
            
        maxprob=np.max(results) #> Select Maximum Probability
        print(f'Maximum Probability : {maxprob}')
        print('')
            
        rest=1-maxprob # Probability of Non-Violence
        diff=maxprob-rest # Difference between Probability of Violence and Non-Violence's
        th=100
            
        if diff>0.80: 
            th=diff # ?? What is supporting basis?
        
        frame_counter=0 #> Initialize frame_counter to 0
        i+=1 #> 1 second elapsed
        
        # When frame_counter>=30, Initialize frame_counter to 0, and repeat above while loop.
                
    # ----- Setting caption option of output video -----
    # Renewed caption is added every 30 frames(if fps=30, it means 1 second.)
    font1=ImageFont.truetype('fonts/Raleway-ExtraBold.ttf', int(0.05*W)) # font option
    font2=ImageFont.truetype('fonts/Raleway-ExtraBold.ttf', int(0.1*W)) #font option
    
    if preds is not None and maxprob is not None:
        if (preds[0][1])<th : #> if violence probability < th, Violence=False (Normal, Green Caption)
            text1_1='Normal'
            text1_2='{:.2f}%'.format(100-(maxprob*100))
            img_pil=Image.fromarray(output)
            draw=ImageDraw.Draw(img_pil)
            draw.text((int(0.025*W), int(0.025*H)), text1_1, font=font1, fill=(0, 255, 0, 0))
            draw.text((int(0.025*W), int(0.095*H)), text1_2, font=font2, fill=(0, 255, 0, 0))
            output=np.array(img_pil)
                
        else : #> if violence probability > th, Violence=True (Violence Alert!, Red Caption)
            text2_1='Violence Alert!'
            text2_2='{:.2f}%'.format(maxprob*100)
            img_pil=Image.fromarray(output)
            draw=ImageDraw.Draw(img_pil)
            draw.text((int(0.025*W), int(0.025*H)), text2_1, font=font1, fill=(0, 0, 255, 0))
            draw.text((int(0.025*W), int(0.095*H)), text2_2, font=font2, fill=(0, 0, 255, 0))
            output=np.array(img_pil)
        
    # Save captioned video file by using 'writer'
    if writer is None:
        fourcc=cv2.VideoWriter_fourcc(*'DIVX')
        writer=cv2.VideoWriter(output_path, fourcc, 30, (W, H), True)
            
    cv2.imshow('This is output', output) # View output in new Window.
    writer.write(output) # Save output in output_path
        
    key=cv2.waitKey(round(1000/fps)) # time gap of frame and next frame
    if key==27: # If you press ESC key, While loop will be breaked and output file will be saved.
        print('ESC is pressed. Video recording ends.')
        break
    
print('Video recording ends. Release Memory.') # Output file will be saved.
writer.release()
vid.release()
cv2.destroyAllWindows()

fps : 30.0


C:\Users\User\AppData\Local\Temp/ipykernel_8956/287044024.py:11: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  video_frm_ar=np.zeros((1, int(fps), 160, 160, 3), dtype=np.float) #frames


preds:[[0.00250655 0.9974935 ]]
Results = [[nan nan]]
Maximum Probability : nan



C:\Users\User\AppData\Local\Temp/ipykernel_8956/287044024.py:55: RuntimeWarning: Mean of empty slice.
  results=np.array(Q)[:i].mean(axis=0)
C:\Users\User\anaconda3\lib\site-packages\numpy\core\_methods.py:180: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


preds:[[0.38167238 0.6183276 ]]
Results = [[0.00250655 0.9974935 ]]
Maximum Probability : 0.9974935054779053

preds:[[0.59665024 0.40334967]]
Results = [[0.19208947 0.80791056]]
Maximum Probability : 0.8079105615615845

preds:[[0.9682635 0.0317365]]
Results = [[0.32694307 0.6730569 ]]
Maximum Probability : 0.6730569005012512

preds:[[0.9926807 0.0073192]]
Results = [[0.48727316 0.51272684]]
Maximum Probability : 0.5127268433570862

There is no frame. Streaming ends.
Video recording ends. Release Memory.
